### Importing function to load city table

In [0]:
def load_city_table():
    # importing required libraries
    from pyspark.sql.functions import col
    
    # specifying json import parameters
    city_info_location = "dbfs:/FileStore/city_list_json.gz"
    options = {"multiLine" : True}
    
    # importing the json data and selecting only the required fields and required rows
    city_df = spark.read.json(city_info_location, **options)
    flattened_city_df = city_df.select('coord.*', '*').select("id", "name", "country", "lat", "lon")
    flattened_city_df = flattened_city_df.filter(col('country') == 'NP')
    flattened_city_df = flattened_city_df.withColumn('id', col('id').cast('integer'))
    
    # renaming some fields to improve redability
    rename_mapping = {
    "lat" : "Latitude",
    "lon" : "Longitude"
    }
    for old, new in rename_mapping.items():
        flattened_city_df = flattened_city_df.withColumnRenamed(old, new)
        
    # loading the city data to dim_city_table
    flattened_city_df.write.format("DELTA").mode('overwrite').saveAsTable('dim_city_table')